# Creating Lightgbm model to predict Doctor's Consultant fee

In [1]:
# Importing fastai libraries

from fastai.imports import *
from fastai.structured import *

### reading feather files which we save in previous notebook

In [2]:
train=pd.read_feather('train_raw')
test=pd.read_feather('test_raw')

In [3]:
train.head(5)

,Qualification,Place,Profile,Fees,experience,information,ratting,new
0,"BHMS, MD - Homeopathy","Kakkanad, Ernakulam",Homeopath,100,24,16.0,100.0,46.666667
1,"BAMS, MD - Ayurveda Medicine","Whitefield, Bangalore",Ayurveda,350,12,76.0,98.0,62.000000
2,"MBBS, MS - Otorhinolaryngology","Mathikere - BEL, Bangalore",ENT Specialist,300,9,NaN,NaN,NaN
3,"BSc - Zoology, BAMS","Bannerghatta Road, Bangalore",Ayurveda,250,12,NaN,NaN,NaN
4,BAMS,"Keelkattalai, Chennai",Ayurveda,250,20,NaN,100.0,NaN


In [4]:
test.head(5)

,Qualification,Place,Profile,experience,information,ratting,new
0,MBBS,"Ghatkopar East, Mumbai",General Medicine,35,NaN,NaN,NaN
1,"MBBS, Diploma in Otorhinolaryngology (DLO)","West Marredpally, Hyderabad",ENT Specialist,31,NaN,NaN,NaN
2,"MBBS, DDVL","KK Nagar, Chennai",Dermatologists,40,NaN,70.0,NaN
3,BAMS,"New Ashok Nagar, Delhi",Ayurveda,0,NaN,NaN,NaN
4,"BDS, MDS - Conservative Dentistry & Endodontics","Kanakpura Road, Bangalore",Dentist,16,NaN,100.0,NaN


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 8 columns):
Qualification    5961 non-null category
Place            5936 non-null category
Profile          5961 non-null category
Fees             5961 non-null int64
experience       5961 non-null int64
information      969 non-null float64
ratting          2659 non-null float64
new              969 non-null float64
dtypes: category(3), float64(3), int64(2)
memory usage: 360.2 KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 7 columns):
Qualification    1987 non-null category
Place            1981 non-null category
Profile          1987 non-null category
experience       1987 non-null int64
information      367 non-null float64
ratting          897 non-null float64
new              367 non-null float64
dtypes: category(3), float64(3), int64(1)
memory usage: 121.6 KB


In [7]:
df_trn, y_trn, nas = proc_df(train, 'Fees', max_n_cat=7)
df_tst, y_tst, nas = proc_df(test, max_n_cat=7)

In [8]:
df_trn.head(5)

,Qualification,Place,experience,information,ratting,new,information_na,ratting_na,new_na,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath,Profile_nan
0,402,300,24,16.0,100.0,46.666667,False,False,False,0,0,0,0,0,1,0
1,74,867,12,76.0,98.0,62.000000,False,False,False,1,0,0,0,0,0,0
2,1136,458,9,26.0,97.0,47.000000,True,True,True,0,0,0,1,0,0,0
3,465,69,12,26.0,97.0,47.000000,True,True,True,1,0,0,0,0,0,0
4,11,338,20,26.0,100.0,47.000000,True,False,True,1,0,0,0,0,0,0


In [9]:
df_tst.head(5)

,Qualification,Place,experience,information,ratting,new,information_na,ratting_na,new_na,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath,Profile_nan
0,242,143,35,23.0,97.0,46.0,True,True,True,0,0,0,0,1,0,0
1,341,581,31,23.0,97.0,46.0,True,True,True,0,0,0,1,0,0,0
2,259,213,40,23.0,70.0,46.0,True,False,True,0,0,1,0,0,0,0
3,7,375,0,23.0,97.0,46.0,True,True,True,1,0,0,0,0,0,0
4,80,232,16,23.0,100.0,46.0,True,False,True,0,1,0,0,0,0,0


In [10]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_trn, y_trn, test_size = 0.25, random_state = 0)

In [11]:
import lightgbm as lgb

### preparing dataset for the lgbm model 

In [12]:
d_train = lgb.Dataset(X_train, label= y_train)
d_test=lgb.Dataset(X_test, label=y_test, reference=d_train,)

### Defining parameters for the lgbm model

In [13]:
params = {'num_leaves': 25,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': 20,
         'learning_rate': 0.001,
         "boosting": "gbdt",
         "metric": 'rmse',
         "verbosity": -1}


In [14]:
## training the model on given train dataset
m = lgb.train(params,
                       d_train,
                       valid_sets=[d_train, d_test],
                       num_boost_round=5000,
                       early_stopping_rounds=100)

[1]	training's rmse: 190.196	valid_1's rmse: 192.893
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 190.161	valid_1's rmse: 192.861
[3]	training's rmse: 190.126	valid_1's rmse: 192.83
[4]	training's rmse: 190.092	valid_1's rmse: 192.798
[5]	training's rmse: 190.057	valid_1's rmse: 192.767
[6]	training's rmse: 190.022	valid_1's rmse: 192.735
[7]	training's rmse: 189.988	valid_1's rmse: 192.704
[8]	training's rmse: 189.953	valid_1's rmse: 192.672
[9]	training's rmse: 189.919	valid_1's rmse: 192.641
[10]	training's rmse: 189.884	valid_1's rmse: 192.61
[11]	training's rmse: 189.85	valid_1's rmse: 192.578
[12]	training's rmse: 189.816	valid_1's rmse: 192.547
[13]	training's rmse: 189.782	valid_1's rmse: 192.516
[14]	training's rmse: 189.748	valid_1's rmse: 192.485
[15]	training's rmse: 189.714	valid_1's rmse: 192.454
[16]	training's rmse: 189.68	valid_1's rmse: 192.423
[17]	training's rmse: 189.646	valid_1's rmse: 192.392
[18]	training's rmse: 189.612	va

[169]	training's rmse: 185.144	valid_1's rmse: 188.265
[170]	training's rmse: 185.118	valid_1's rmse: 188.242
[171]	training's rmse: 185.092	valid_1's rmse: 188.217
[172]	training's rmse: 185.067	valid_1's rmse: 188.194
[173]	training's rmse: 185.041	valid_1's rmse: 188.171
[174]	training's rmse: 185.016	valid_1's rmse: 188.148
[175]	training's rmse: 184.99	valid_1's rmse: 188.124
[176]	training's rmse: 184.965	valid_1's rmse: 188.102
[177]	training's rmse: 184.939	valid_1's rmse: 188.078
[178]	training's rmse: 184.914	valid_1's rmse: 188.055
[179]	training's rmse: 184.889	valid_1's rmse: 188.032
[180]	training's rmse: 184.863	valid_1's rmse: 188.009
[181]	training's rmse: 184.838	valid_1's rmse: 187.985
[182]	training's rmse: 184.813	valid_1's rmse: 187.963
[183]	training's rmse: 184.788	valid_1's rmse: 187.94
[184]	training's rmse: 184.762	valid_1's rmse: 187.916
[185]	training's rmse: 184.737	valid_1's rmse: 187.893
[186]	training's rmse: 184.712	valid_1's rmse: 187.87
[187]	trainin

[424]	training's rmse: 179.671	valid_1's rmse: 183.34
[425]	training's rmse: 179.653	valid_1's rmse: 183.325
[426]	training's rmse: 179.635	valid_1's rmse: 183.309
[427]	training's rmse: 179.618	valid_1's rmse: 183.295
[428]	training's rmse: 179.601	valid_1's rmse: 183.281
[429]	training's rmse: 179.583	valid_1's rmse: 183.266
[430]	training's rmse: 179.565	valid_1's rmse: 183.251
[431]	training's rmse: 179.548	valid_1's rmse: 183.237
[432]	training's rmse: 179.53	valid_1's rmse: 183.222
[433]	training's rmse: 179.512	valid_1's rmse: 183.207
[434]	training's rmse: 179.495	valid_1's rmse: 183.193
[435]	training's rmse: 179.477	valid_1's rmse: 183.179
[436]	training's rmse: 179.46	valid_1's rmse: 183.164
[437]	training's rmse: 179.442	valid_1's rmse: 183.149
[438]	training's rmse: 179.424	valid_1's rmse: 183.135
[439]	training's rmse: 179.407	valid_1's rmse: 183.12
[440]	training's rmse: 179.39	valid_1's rmse: 183.106
[441]	training's rmse: 179.373	valid_1's rmse: 183.091
[442]	training'

[711]	training's rmse: 175.422	valid_1's rmse: 179.863
[712]	training's rmse: 175.411	valid_1's rmse: 179.854
[713]	training's rmse: 175.399	valid_1's rmse: 179.844
[714]	training's rmse: 175.387	valid_1's rmse: 179.834
[715]	training's rmse: 175.375	valid_1's rmse: 179.824
[716]	training's rmse: 175.362	valid_1's rmse: 179.815
[717]	training's rmse: 175.35	valid_1's rmse: 179.805
[718]	training's rmse: 175.339	valid_1's rmse: 179.796
[719]	training's rmse: 175.326	valid_1's rmse: 179.787
[720]	training's rmse: 175.314	valid_1's rmse: 179.778
[721]	training's rmse: 175.302	valid_1's rmse: 179.769
[722]	training's rmse: 175.291	valid_1's rmse: 179.76
[723]	training's rmse: 175.279	valid_1's rmse: 179.75
[724]	training's rmse: 175.266	valid_1's rmse: 179.742
[725]	training's rmse: 175.255	valid_1's rmse: 179.733
[726]	training's rmse: 175.243	valid_1's rmse: 179.724
[727]	training's rmse: 175.23	valid_1's rmse: 179.715
[728]	training's rmse: 175.217	valid_1's rmse: 179.705
[729]	training

[974]	training's rmse: 172.587	valid_1's rmse: 177.929
[975]	training's rmse: 172.579	valid_1's rmse: 177.924
[976]	training's rmse: 172.57	valid_1's rmse: 177.919
[977]	training's rmse: 172.56	valid_1's rmse: 177.914
[978]	training's rmse: 172.551	valid_1's rmse: 177.909
[979]	training's rmse: 172.542	valid_1's rmse: 177.904
[980]	training's rmse: 172.533	valid_1's rmse: 177.899
[981]	training's rmse: 172.524	valid_1's rmse: 177.895
[982]	training's rmse: 172.514	valid_1's rmse: 177.89
[983]	training's rmse: 172.504	valid_1's rmse: 177.885
[984]	training's rmse: 172.494	valid_1's rmse: 177.881
[985]	training's rmse: 172.485	valid_1's rmse: 177.876
[986]	training's rmse: 172.476	valid_1's rmse: 177.871
[987]	training's rmse: 172.466	valid_1's rmse: 177.865
[988]	training's rmse: 172.457	valid_1's rmse: 177.861
[989]	training's rmse: 172.446	valid_1's rmse: 177.856
[990]	training's rmse: 172.437	valid_1's rmse: 177.852
[991]	training's rmse: 172.428	valid_1's rmse: 177.846
[992]	trainin

...

[2669]	training's rmse: 162.377	valid_1's rmse: 173.484
[2670]	training's rmse: 162.373	valid_1's rmse: 173.484
[2671]	training's rmse: 162.37	valid_1's rmse: 173.482
[2672]	training's rmse: 162.365	valid_1's rmse: 173.48
[2673]	training's rmse: 162.359	valid_1's rmse: 173.478
[2674]	training's rmse: 162.355	valid_1's rmse: 173.476
[2675]	training's rmse: 162.351	valid_1's rmse: 173.475
[2676]	training's rmse: 162.345	valid_1's rmse: 173.473
[2677]	training's rmse: 162.342	valid_1's rmse: 173.471
[2678]	training's rmse: 162.338	valid_1's rmse: 173.47
[2679]	training's rmse: 162.334	valid_1's rmse: 173.468
[2680]	training's rmse: 162.331	valid_1's rmse: 173.468
[2681]	training's rmse: 162.325	valid_1's rmse: 173.466
[2682]	training's rmse: 162.32	valid_1's rmse: 173.464
[2683]	training's rmse: 162.317	valid_1's rmse: 173.463
[2684]	training's rmse: 162.311	valid_1's rmse: 173.461
[2685]	training's rmse: 162.308	valid_1's rmse: 173.459
[2686]	training's rmse: 162.304	valid_1's rmse: 173.

[2822]	training's rmse: 161.726	valid_1's rmse: 173.307
[2823]	training's rmse: 161.722	valid_1's rmse: 173.306
[2824]	training's rmse: 161.719	valid_1's rmse: 173.305
[2825]	training's rmse: 161.715	valid_1's rmse: 173.305
[2826]	training's rmse: 161.712	valid_1's rmse: 173.304
[2827]	training's rmse: 161.707	valid_1's rmse: 173.305
[2828]	training's rmse: 161.703	valid_1's rmse: 173.303
[2829]	training's rmse: 161.699	valid_1's rmse: 173.303
[2830]	training's rmse: 161.696	valid_1's rmse: 173.303
[2831]	training's rmse: 161.692	valid_1's rmse: 173.303
[2832]	training's rmse: 161.687	valid_1's rmse: 173.304
[2833]	training's rmse: 161.683	valid_1's rmse: 173.302
[2834]	training's rmse: 161.68	valid_1's rmse: 173.302
[2835]	training's rmse: 161.675	valid_1's rmse: 173.302
[2836]	training's rmse: 161.671	valid_1's rmse: 173.3
[2837]	training's rmse: 161.668	valid_1's rmse: 173.3
[2838]	training's rmse: 161.663	valid_1's rmse: 173.301
[2839]	training's rmse: 161.659	valid_1's rmse: 173.2

[3125]	training's rmse: 160.632	valid_1's rmse: 173.193
[3126]	training's rmse: 160.628	valid_1's rmse: 173.194
[3127]	training's rmse: 160.625	valid_1's rmse: 173.193
[3128]	training's rmse: 160.621	valid_1's rmse: 173.194
[3129]	training's rmse: 160.617	valid_1's rmse: 173.193
[3130]	training's rmse: 160.613	valid_1's rmse: 173.192
[3131]	training's rmse: 160.609	valid_1's rmse: 173.193
[3132]	training's rmse: 160.605	valid_1's rmse: 173.192
[3133]	training's rmse: 160.601	valid_1's rmse: 173.193
[3134]	training's rmse: 160.598	valid_1's rmse: 173.193
[3135]	training's rmse: 160.594	valid_1's rmse: 173.193
[3136]	training's rmse: 160.591	valid_1's rmse: 173.193
[3137]	training's rmse: 160.587	valid_1's rmse: 173.191
[3138]	training's rmse: 160.583	valid_1's rmse: 173.192
[3139]	training's rmse: 160.579	valid_1's rmse: 173.191
[3140]	training's rmse: 160.575	valid_1's rmse: 173.192
[3141]	training's rmse: 160.572	valid_1's rmse: 173.192
[3142]	training's rmse: 160.568	valid_1's rmse: 

[3276]	training's rmse: 160.097	valid_1's rmse: 173.162
[3277]	training's rmse: 160.093	valid_1's rmse: 173.161
[3278]	training's rmse: 160.09	valid_1's rmse: 173.161
[3279]	training's rmse: 160.085	valid_1's rmse: 173.16
[3280]	training's rmse: 160.082	valid_1's rmse: 173.159
[3281]	training's rmse: 160.078	valid_1's rmse: 173.157
[3282]	training's rmse: 160.075	valid_1's rmse: 173.157
[3283]	training's rmse: 160.07	valid_1's rmse: 173.155
[3284]	training's rmse: 160.067	valid_1's rmse: 173.156
[3285]	training's rmse: 160.064	valid_1's rmse: 173.155
[3286]	training's rmse: 160.06	valid_1's rmse: 173.154
[3287]	training's rmse: 160.056	valid_1's rmse: 173.154
[3288]	training's rmse: 160.053	valid_1's rmse: 173.152
[3289]	training's rmse: 160.05	valid_1's rmse: 173.153
[3290]	training's rmse: 160.046	valid_1's rmse: 173.152
[3291]	training's rmse: 160.042	valid_1's rmse: 173.151
[3292]	training's rmse: 160.038	valid_1's rmse: 173.149
[3293]	training's rmse: 160.035	valid_1's rmse: 173.1

[3430]	training's rmse: 159.543	valid_1's rmse: 173.072
[3431]	training's rmse: 159.54	valid_1's rmse: 173.071
[3432]	training's rmse: 159.537	valid_1's rmse: 173.071
[3433]	training's rmse: 159.533	valid_1's rmse: 173.071
[3434]	training's rmse: 159.53	valid_1's rmse: 173.071
[3435]	training's rmse: 159.526	valid_1's rmse: 173.07
[3436]	training's rmse: 159.523	valid_1's rmse: 173.07
[3437]	training's rmse: 159.52	valid_1's rmse: 173.07
[3438]	training's rmse: 159.516	valid_1's rmse: 173.071
[3439]	training's rmse: 159.513	valid_1's rmse: 173.071
[3440]	training's rmse: 159.51	valid_1's rmse: 173.071
[3441]	training's rmse: 159.506	valid_1's rmse: 173.07
[3442]	training's rmse: 159.503	valid_1's rmse: 173.07
[3443]	training's rmse: 159.5	valid_1's rmse: 173.07
[3444]	training's rmse: 159.496	valid_1's rmse: 173.07
[3445]	training's rmse: 159.493	valid_1's rmse: 173.069
[3446]	training's rmse: 159.489	valid_1's rmse: 173.069
[3447]	training's rmse: 159.486	valid_1's rmse: 173.069
[3448

In [15]:
predictions=m.predict(X_test)

In [16]:
predictions[:10]

array([208.62126, 256.3972 , 252.56681, 347.39445, 284.00353, 360.31079, 216.3097 , 323.65013, 291.65138,
       281.37444])

In [17]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_test), y_test)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [18]:
print_score(m)

[159.432300932583, 173.06613046420935]


### score has improved since last model of random forest and hence much accurate predictions will be made using lgbm model.